# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy.stats import percentileofscore as score
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'EEA( TMLGTRA/ASC BLKSSQGN)N ADEOL', 'calculationPrice': 'close', 'open': 129.35, 'openTime': 1655355193389, 'openSource': 'lffiiaoc', 'close': 126.76, 'closeTime': 1649618673189, 'closeSource': 'ociaiffl', 'high': 124.61, 'highTime': 1692266619950, 'highSource': 'dpn5e creul1yitdi ema e', 'low': 122.85, 'lowTime': 1660003786207, 'lowSource': 'nleir1itd dep eea5 yumc', 'latestPrice': 123.15, 'latestSource': 'Close', 'latestTime': 'March 23, 2021', 'latestUpdate': 1662733025510, 'latestVolume': 97362507, 'iexRealtimePrice': 125.08, 'iexRealtimeSize': 31, 'iexLastUpdated': 1659827739626, 'delayedPrice': 126.74, 'delayedPriceTime': 1629304288345, 'oddLotDelayedPrice': 128.23, 'oddLotDelayedPriceTime': 1621979177955, 'extendedPrice': 124.37, 'extendedChange': -0.09, 'extendedChangePercent': -0.00076, 'extendedPriceTime': 1658839588279, 'previousClose': 127.35, 'previousVolume': 114977799, 'change': -0.88, 'changePercent': -0

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

33.8

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['peRatio'],
            'N/A'
        ], index = my_columns
        ),
            ignore_index=True
        )
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,122.24,48.67,N/A
1,AAL,23.08,-1.23,N/A
2,AAP,183.56,26.07,N/A
3,AAPL,128.22,34.5,N/A
4,ABBV,105.48,39.29,N/A
...,...,...,...,...
500,YUM,110.00,37.57,N/A
501,ZBH,162.07,-242.4,N/A
502,ZBRA,480.43,51.87,N/A
503,ZION,55.42,8.47,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = False, inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe [:50]
final_dataframe.reset_index(inplace=True, drop=True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BKNG,2284.00,1562.57,N/A
1,SBAC,281.00,1297.74,N/A
2,NOW,481.73,823.14,N/A
3,GILD,67.50,693.46,N/A
4,FIS,146.75,565.07,N/A
5,AES,26.81,385.37,N/A
6,TEL,129.38,368.64,N/A
7,KMI,16.13,338.18,N/A
8,UDR,44.90,216.54,N/A
9,REG,55.97,209.65,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [7]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [8]:
portfolio_input()

Enter the value of your portfolio: 1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [9]:
position_size = float(portfolio_size)/len(final_dataframe.index)

for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Number of Shares to Buy'])
    
final_dataframe

TypeError: unsupported operand type(s) for /: 'float' and 'str'

## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [10]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
#print(data['AAPL']['advanced-stats'])

# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data['AAPL']['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data['AAPL']['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data['AAPL']['advanced-stats']['enterpriseValue']
ebitda = data['AAPL']['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data['AAPL']['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [11]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda= enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN

        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
            
        rv_dataframe = rv_dataframe.append(
        pd.Series([
            symbol,
            data[symbol]['quote']['latestPrice'],
            'N/A',
            data[symbol]['quote']['peRatio'],
            'N/A',
            data[symbol]['advanced-stats']['priceToBook'],
            'N/A',
            data[symbol]['advanced-stats']['priceToSales'],
            'N/A',
            ev_to_ebitda,
            'N/A',
            ev_to_gross_profit,
            'N/A',
            'N/A'
        ],
        index = rv_columns),
            ignore_index=True)

In [12]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,127.43,N/A,47.08,N/A,8.1,N/A,6.85,N/A,30.132613,N/A,13.053509,N/A,N/A
1,AAL,22.57,N/A,-1.27,N/A,-2.12,N/A,0.8512,N/A,-4.359845,N/A,2.310935,N/A,N/A
2,AAP,188.41,N/A,26.06,N/A,3.3,N/A,1.17,N/A,11.923709,N/A,2.684088,N/A,N/A
3,AAPL,123.11,N/A,34.8,N/A,32.42,N/A,7.07,N/A,24.644365,N/A,18.450192,N/A,N/A
4,ABBV,105.80,N/A,39.31,N/A,14.38,N/A,4.21,N/A,14.051232,N/A,8.796424,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,112.00,N/A,37.14,N/A,-4.14,N/A,5.77,N/A,23.670777,N/A,9.849196,N/A,N/A
501,ZBH,160.59,N/A,-237.6,N/A,2.72,N/A,4.68,N/A,29.887737,N/A,8.143634,N/A,N/A
502,ZBRA,465.99,N/A,51.69,N/A,11.56,N/A,5.73,N/A,34.295464,N/A,13.045851,N/A,N/A
503,ZION,54.31,N/A,8.82,N/A,1.2,N/A,2.6,N/A,6.728785,N/A,2.813378,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [13]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,224.040,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
42,APA,18.793,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,253.800,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.89,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,65.240,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.470,N/A,14.44,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,60.120,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,39.980,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,172.280,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2150.410,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [14]:
for column in ['Price-to-Earnings Ratio','Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [15]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [34]:
metrics = {
    'Price-to-Earnings Ratio':'PE Percentile',
    'Price-to-Book Ratio':'PB Percentile',
    'Price-to-Sales Ratio':'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/GP':'EV/GP Percentile'
}
for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]]= score(rv_dataframe[metric],rv_dataframe.loc[row, metric])/100
        
        
rv_dataframe

C:\Users\vyle4\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,27.39,1825,6.87,0.44,0.5147,0.14,0.4182,0.18,3.425219,0.16,0.402819,0.04,5.74257
1,MCK,192.15,260,-7.76,0.24,-63.0200,0.02,0.1291,0.04,7.459850,0.42,2.774655,0.66,6.25743
2,FTI,7.51,6657,-1.05,0.4,0.8359,0.26,0.2623,0.08,2.164834,0.12,1.352887,0.3,6.41584
3,AIG,46.76,1069,-6.70,0.28,0.5974,0.16,0.9333,0.46,7.070628,0.36,0.898549,0.14,6.81188
4,MET,60.58,825,10.56,0.56,0.7135,0.18,0.7672,0.36,5.122419,0.2,0.761081,0.1,7.44554
5,AAL,22.57,2215,-1.27,0.38,-2.1200,0.12,0.8512,0.44,-4.359845,0.08,2.310935,0.54,8.11881
6,ALL,115.85,431,6.61,0.42,1.3000,0.5,0.7741,0.38,2.419169,0.14,0.732774,0.08,8.27723
7,DXC,26.96,1854,-2.38,0.34,1.2500,0.46,0.3740,0.14,5.420065,0.22,2.456774,0.6,8.63366
8,F,12.58,3974,-38.39,0.08,1.5900,0.72,0.3860,0.16,6.281487,0.26,2.874155,0.7,8.75248
9,ABC,117.99,423,-7.30,0.26,-33.9000,0.04,0.1203,0.02,8.604292,0.72,4.125204,0.84,9.26733


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [35]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

rv_dataframe

C:\Users\vyle4\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,27.39,1825,6.87,0.44,0.5147,0.14,0.4182,0.18,3.425219,0.16,0.402819,0.04,0.192
1,MCK,192.15,260,-7.76,0.24,-63.0200,0.02,0.1291,0.04,7.459850,0.42,2.774655,0.66,0.276
2,FTI,7.51,6657,-1.05,0.4,0.8359,0.26,0.2623,0.08,2.164834,0.12,1.352887,0.3,0.232
3,AIG,46.76,1069,-6.70,0.28,0.5974,0.16,0.9333,0.46,7.070628,0.36,0.898549,0.14,0.28
4,MET,60.58,825,10.56,0.56,0.7135,0.18,0.7672,0.36,5.122419,0.2,0.761081,0.1,0.28
5,AAL,22.57,2215,-1.27,0.38,-2.1200,0.12,0.8512,0.44,-4.359845,0.08,2.310935,0.54,0.312
6,ALL,115.85,431,6.61,0.42,1.3000,0.5,0.7741,0.38,2.419169,0.14,0.732774,0.08,0.304
7,DXC,26.96,1854,-2.38,0.34,1.2500,0.46,0.3740,0.14,5.420065,0.22,2.456774,0.6,0.352
8,F,12.58,3974,-38.39,0.08,1.5900,0.72,0.3860,0.16,6.281487,0.26,2.874155,0.7,0.384
9,ABC,117.99,423,-7.30,0.26,-33.9000,0.04,0.1203,0.02,8.604292,0.72,4.125204,0.84,0.376


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [36]:
rv_dataframe.sort_values('RV Score', ascending = True, inplace=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(inplace=True, drop=True)
rv_dataframe

<ipython-input-36-9a00e5ef61d1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_dataframe.sort_values('RV Score', ascending = True, inplace=True)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,27.39,1825,6.87,0.44,0.5147,0.14,0.4182,0.18,3.425219,0.16,0.402819,0.04,0.192
1,FTI,7.51,6657,-1.05,0.4,0.8359,0.26,0.2623,0.08,2.164834,0.12,1.352887,0.3,0.232
2,BA,251.58,198,-12.04,0.18,-8.0000,0.1,2.4400,0.92,-16.728480,0.04,-31.443671,0.02,0.252
3,MCK,192.15,260,-7.76,0.24,-63.0200,0.02,0.1291,0.04,7.459850,0.42,2.774655,0.66,0.276
4,AIG,46.76,1069,-6.70,0.28,0.5974,0.16,0.9333,0.46,7.070628,0.36,0.898549,0.14,0.28
5,MET,60.58,825,10.56,0.56,0.7135,0.18,0.7672,0.36,5.122419,0.2,0.761081,0.1,0.28
6,ALL,115.85,431,6.61,0.42,1.3000,0.5,0.7741,0.38,2.419169,0.14,0.732774,0.08,0.304
7,AAL,22.57,2215,-1.27,0.38,-2.1200,0.12,0.8512,0.44,-4.359845,0.08,2.310935,0.54,0.312
8,DXC,26.96,1854,-2.38,0.34,1.2500,0.46,0.3740,0.14,5.420065,0.22,2.456774,0.6,0.352
9,ABC,117.99,423,-7.30,0.26,-33.9000,0.04,0.1203,0.02,8.604292,0.72,4.125204,0.84,0.376


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [19]:
portfolio_input()

Enter the value of your portfolio: 2500000


In [37]:
position_size = float(portfolio_size)/len(rv_dataframe.index)
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row, 'Price'])

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,27.39,1825,6.87,0.44,0.5147,0.14,0.4182,0.18,3.425219,0.16,0.402819,0.04,0.192
1,FTI,7.51,6657,-1.05,0.4,0.8359,0.26,0.2623,0.08,2.164834,0.12,1.352887,0.3,0.232
2,BA,251.58,198,-12.04,0.18,-8.0000,0.1,2.4400,0.92,-16.728480,0.04,-31.443671,0.02,0.252
3,MCK,192.15,260,-7.76,0.24,-63.0200,0.02,0.1291,0.04,7.459850,0.42,2.774655,0.66,0.276
4,AIG,46.76,1069,-6.70,0.28,0.5974,0.16,0.9333,0.46,7.070628,0.36,0.898549,0.14,0.28
5,MET,60.58,825,10.56,0.56,0.7135,0.18,0.7672,0.36,5.122419,0.2,0.761081,0.1,0.28
6,ALL,115.85,431,6.61,0.42,1.3000,0.5,0.7741,0.38,2.419169,0.14,0.732774,0.08,0.304
7,AAL,22.57,2215,-1.27,0.38,-2.1200,0.12,0.8512,0.44,-4.359845,0.08,2.310935,0.54,0.312
8,DXC,26.96,1854,-2.38,0.34,1.2500,0.46,0.3740,0.14,5.420065,0.22,2.456774,0.6,0.352
9,ABC,117.99,423,-7.30,0.26,-33.9000,0.04,0.1203,0.02,8.604292,0.72,4.125204,0.84,0.376


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [38]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine = 'xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [39]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )
float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [40]:
column_formats = {
        'A': ['Ticker', string_template],
        'B': ['Price', dollar_template],
        'C': ['Number of Shares to Buy', integer_template],
        'D': ['Price-to-Earnings Ratio', float_template],
        'E': ['PE Percentile', percent_template],
        'F': ['Price-to-Book Ratio', float_template],
        'G': ['PB Percentile',percent_template],
        'H': ['Price-to-Sales Ratio', float_template],
        'I': ['PS Percentile', percent_template],
        'J': ['EV/EBITDA', float_template],
        'K': ['EV/EBITDA Percentile', percent_template],
        'L': ['EV/GP', float_template],
        'M': ['EV/GP Percentile', percent_template],
        'N': ['RV Score', percent_template]
     }


for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [42]:
writer.save()